In [56]:
# importing libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from joblib import dump
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.metrics import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError
from math import sqrt
import numpy as np
import scipy.stats as stats

In [57]:
# importing the data
data = pd.read_csv("modeling_data_2.csv")
data.head()

,Date,Year,Year_sin,Year_cos,Wholesale_lag_7,Retail_lag_7,Wholesale_rolling_mean_7d,Retail_rolling_mean_7d,Wholesale_rolling_std_7d,Retail_rolling_std_7d,Wholesale,Retail
0,2021-05-24,2021,0.0,1.0,27.78,40.0,0.115639,0.142735,0.016814,0.000000,0.041655,0.062485
1,2021-05-24,2021,0.0,1.0,27.78,40.0,0.104055,0.142735,0.016814,0.000000,0.033315,0.062485
2,2021-06-28,2021,0.0,1.0,27.78,40.0,0.112542,0.148688,0.019208,0.008545,0.046650,0.070298
3,2021-09-20,2021,0.0,1.0,27.78,40.0,0.116785,0.140056,0.017940,0.015917,0.046650,0.049985
4,2021-10-04,2021,0.0,1.0,27.78,40.0,0.118406,0.137634,0.015975,0.014497,0.044985,0.056017


In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15493 entries, 0 to 15492
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       15493 non-null  object 
 1   Year                       15493 non-null  int64  
 2   Year_sin                   15493 non-null  float64
 3   Year_cos                   15493 non-null  float64
 4   Wholesale_lag_7            15491 non-null  float64
 5   Retail_lag_7               15491 non-null  float64
 6   Wholesale_rolling_mean_7d  15493 non-null  float64
 7   Retail_rolling_mean_7d     15493 non-null  float64
 8   Wholesale_rolling_std_7d   15493 non-null  float64
 9   Retail_rolling_std_7d      15493 non-null  float64
 10  Wholesale                  15493 non-null  float64
 11  Retail                     15493 non-null  float64
dtypes: float64(10), int64(1), object(1)
memory usage: 1.4+ MB


In [59]:
data.drop(columns=["Date"], inplace=True)

In [60]:
data.isna().sum()

Year                         0
Year_sin                     0
Year_cos                     0
Wholesale_lag_7              2
Retail_lag_7                 2
Wholesale_rolling_mean_7d    0
Retail_rolling_mean_7d       0
Wholesale_rolling_std_7d     0
Retail_rolling_std_7d        0
Wholesale                    0
Retail                       0
dtype: int64

In [62]:
features = data.drop(columns=["Wholesale", "Retail"])
target1 = data["Wholesale"]
target2 = data[ "Retail"]

In [72]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train_wholesale, X_test_wholesale, y_train_wholesale, y_test_wholesale = train_test_split(features, target1 , test_size=0.2, random_state=3000)

print(f"Training set shape: {X_train_wholesale.shape}")
print(f"Testing set shape: {X_test_wholesale.shape}")


Training set shape: (12394, 9)
Testing set shape: (3099, 9)


In [74]:
# Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit the scaler on the training data (excluding 'Date' and 'Year' if necessary)
columns_to_exclude = ['Year']

# Select columns to scale
X_train_to_scale = X_train_wholesale.drop(columns=columns_to_exclude)
X_test_to_scale = X_test_wholesale.drop(columns=columns_to_exclude)

# Fit and transform the training data
X_train_scaled = scaler_X.fit_transform(X_train_to_scale)
y_train_scaled = scaler_y.fit_transform(y_train_wholesale.values.reshape(-1, 1))

# Transform the test data using the fitted scaler
X_test_scaled = scaler_X.transform(X_test_to_scale)
y_test_scaled = scaler_y.transform(y_test_wholesale.values.reshape(-1, 1))

# Optionally, add back the excluded columns if you need them
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_to_scale.columns, index=X_train_wholesale.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_to_scale.columns, index=X_test_wholesale.index)

# If you want to keep the Date and Year columns for reference (but not scaled), add them back to the scaled data
X_train_scaled = pd.concat([X_train_wholesale[columns_to_exclude], X_train_scaled], axis=1)
X_test_scaled = pd.concat([X_test_wholesale[columns_to_exclude], X_test_scaled], axis=1)

# Now your data is ready for the model
print(f"Scaled X_train_wholesale shape: {X_train_scaled.shape}")
print(f"Scaled y_train_wholesale shape: {y_train_scaled.shape}")


Scaled X_train_wholesale shape: (12394, 10)
Scaled y_train_wholesale shape: (12394, 1)


In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# # Specify the columns to exclude
# exclude_columns = ['Year', 'Wholesale', 'Retail']

# # Separate the columns to be scaled and the excluded columns
# columns_to_scale = .drop(columns=exclude_columns).columns
# data_to_scale = data[columns_to_scale]

# # Initialize MinMaxScaler
# scaler = MinMaxScaler()

# # Apply MinMaxScaler to the columns that need scaling
# data_scaled = pd.DataFrame(scaler.fit_transform(data_to_scale), columns=columns_to_scale)

# # Recombine the scaled columns with the excluded columns (Date, Year)
# data = pd.concat([data[exclude_columns].reset_index(drop=True), data_scaled], axis=1)

# # Check the resulting scaled dataset
# data.head()


In [75]:
# Parameters
time_steps = 10  # Adjust this as needed

# Assume X_train_wholesale and y_train_wholesale are already scaled using MinMaxScaler
X_train_wholesale_scaled = np.array(X_train_scaled)  # Your scaled features
y_train_wholesale_scaled = np.array(y_train_scaled)  # Your scaled target (reshaped)

# Initialize lists to hold reshaped data
X_lstm_wholesale = []
y_lstm_wholesale = []

# Loop over the scaled data to create sequences for LSTM
for i in range(len(X_train_wholesale_scaled) - time_steps):
    # For each step, grab the previous 'time_steps' of features
    X_lstm_wholesale.append(X_train_wholesale_scaled[i:i + time_steps])
    
    # Grab the corresponding target value (Wholesale) at the next time step
    y_lstm_wholesale.append(y_train_wholesale_scaled[i + time_steps])

# Convert lists to NumPy arrays
X_lstm_wholesale = np.array(X_lstm_wholesale)
y_lstm_wholesale = np.array(y_lstm_wholesale)

# Check the shape of the reshaped data
print(f"Shape of X_lstm_wholesale: {X_lstm_wholesale.shape}")  # Should be (samples, time_steps, features)
print(f"Shape of y_lstm_wholesale: {y_lstm_wholesale.shape}")  # Should be (samples, 1)


Shape of X_lstm_wholesale: (12384, 10, 10)
Shape of y_lstm_wholesale: (12384, 1)


In [76]:
X_train_wholesale = np.array(X_train_wholesale).astype('float32')
y_train_wholesale = np.array(y_train_wholesale).astype('float32')
X_test_wholesale = np.array(X_test_wholesale).astype('float32')
y_test_wholesale = np.array(y_test_wholesale).astype('float32')

In [79]:
# Check the shape of X_train_wholesale
print(X_train_wholesale.shape)  # This should be (samples, time_steps, features)

# Reshape if necessary to ensure it's 3D
if len(X_train_wholesale.shape) == 2:
    # Reshape to 3D: (samples, time_steps, features)
    X_train_wholesale = .reshape((X_lstm_wholesale.shape[0], time_steps, -1))

# Now define the model
model = Sequential()
model.add(LSTM(50, reX_train_wholesaleturn_sequences=True, activation="tanh", input_shape=(X_train_wholesale.shape[1], X_train_wholesale.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))  # If predicting only one value (Wholesale or Retail)


(12394, 9)


ValueError: cannot reshape array of size 111546 into shape (12394,10,newaxis)

In [78]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True,  activation="tanh", input_shape=(X_lstm_wholesale.shape[1], X_train_wholesale.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))  # 2 output neurons for Wholesale and Retail prices


IndexError: tuple index out of range

In [67]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


In [68]:
history = model.fit(X_train_wholesale, y_train_wholesale, epochs=6, batch_size= 20, validation_data=(X_test_wholesale,y_test_wholesale), verbose=1)


Epoch 1/6
620/620 [==============================] - 89s 31ms/step - loss: nan - val_loss: nan
Epoch 2/6
620/620 [==============================] - 18s 29ms/step - loss: nan - val_loss: nan
Epoch 3/6
620/620 [==============================] - 16s 26ms/step - loss: nan - val_loss: nan
Epoch 4/6
620/620 [==============================] - 14s 23ms/step - loss: nan - val_loss: nan
Epoch 5/6
620/620 [==============================] - 14s 22ms/step - loss: nan - val_loss: nan
Epoch 6/6
620/620 [==============================] - 17s 27ms/step - loss: nan - val_loss: nan


In [69]:
# Evaluate the model on test data
test_loss = model.evaluate(X_test_wholesale, y_test_wholesale)
print(f"Test Loss: {test_loss}")

# Make predictions
y_pred = model.predict(X_test_wholesale)

# Compare predictions with actual values
print("Predicted Prices:", y_pred[:5])
print("Actual Prices:", y_test_wholesale[:5])

97/97 [==============================] - 1s 12ms/step - loss: nan
Test Loss: nan
97/97 [==============================] - 4s 11ms/step
Predicted Prices: [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Actual Prices: [0.11665617 0.07498575 0.04198542 0.08164582 0.0499805 ]


In [70]:
# def plot_predictions(test, predicted):
#     plt.plot(test, color="gray", label="Real")
#     plt.plot(predicted, color="red", label="Predicted")
#     plt.title("MasterCard Stock Price Prediction")
#     plt.xlabel("Time")
#     plt.ylabel("MasterCard Stock Price")
#     plt.legend()
#     plt.show()


# def return_rmse(test, predicted):
#     rmse = np.sqrt(mean_squared_error(test, predicted))
#     print("The root mean squared error is {:.2f}.".format(rmse))


In [71]:
X_train_wholesale

array([[[2.02400000e+03, 5.00000000e-01, 1.00000000e+00, ...,
         1.59323782e-01, 8.01830832e-03, 1.37214325e-02],
        [2.02400000e+03, 5.00000000e-01, 1.00000000e+00, ...,
         1.57201022e-01, 8.01830832e-03, 1.39832934e-02],
        [2.02400000e+03, 5.00000000e-01, 1.00000000e+00, ...,
         1.55073181e-01, 8.01830832e-03, 1.56933572e-02],
        ...,
        [2.02400000e+03, 5.00000000e-01, 1.00000000e+00, ...,
         1.40188396e-01, 1.01063028e-02, 6.59764698e-03],
        [2.02100000e+03, 5.00000000e-01, 1.00000000e+00, ...,
         2.49892846e-01, 6.71343552e-03, 1.67441089e-02],
        [2.02100000e+03, 5.00000000e-01, 1.00000000e+00, ...,
         1.42734677e-01, 6.71343552e-03, 1.67441089e-02]],

       [[2.02200000e+03, 1.00000000e+00, 4.44089210e-16, ...,
         1.24874979e-01, 1.00701535e-02, 2.09301356e-02],
        [2.02300000e+03, 0.00000000e+00, 0.00000000e+00, ...,
         1.74786881e-01, 1.12386614e-01, 7.31514767e-02],
        [2.02100000e+03, 